# **Stacking from scratch**

Berikan penjelasan terkait Stacking!

Source : https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/
Youtube : https://www.youtube.com/watch?v=a4IS1Ai7GCI

Stacking atau stacked generalization adalah teknik ensemble learning yang menggabungkan beberapa model dasar (base models) melalui satu model meta (meta-model) untuk meningkatkan akurasi prediksi. Tujuannya adalah memanfaatkan kelebihan tiap model dan mengurangi kelemahannya sehingga hasil prediksi menjadi lebih baik.

Proses stacking terdiri dari dua lapisan. Lapisan pertama berisi base models seperti Decision Tree, KNN, Logistic Regression, atau SVM yang belajar langsung dari data pelatihan. Lapisan kedua adalah meta model yang menerima hasil prediksi dari model-model dasar sebagai input dan menghasilkan prediksi akhir. Biasanya, meta model berupa model sederhana seperti Logistic Regression agar tidak mudah overfitting.

Cara kerja stacking dilakukan dengan melatih model dasar pada data pelatihan, lalu menghasilkan prediksi pada data yang tidak digunakan untuk pelatihan (out-of-fold prediction). Prediksi ini kemudian dijadikan data baru untuk melatih meta model. Saat melakukan prediksi, base models memberikan hasil awal yang digabung oleh meta model untuk menghasilkan keputusan akhir.

Keunggulan stacking adalah mampu meningkatkan performa dan stabilitas prediksi karena mengombinasikan berbagai algoritma. Namun, teknik ini lebih kompleks, memakan waktu pelatihan lebih lama, dan berisiko overfitting jika tidak diatur dengan baik.

Menurut GeeksforGeeks, implementasi stacking dapat dilakukan menggunakan StackingClassifier dari scikit-learn dengan contoh model KNN dan Naive Bayes sebagai base models, serta Logistic Regression sebagai meta model. Hasilnya menunjukkan peningkatan akurasi dibanding model tunggal, menjadikan stacking salah satu metode ensemble yang efektif dalam machine learning.

In [1]:
# Import Libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier


**Pendefinisian class stack**

Bertujuan untuk membuat kerangka kerja ensemble learning berbasis stacking dan blending secara modular dan fleksibel. Class ini memungkinkan pengguna untuk menggabungkan beberapa model dasar (base learners) dan satu model meta (final estimator) untuk menghasilkan prediksi akhir yang lebih akurat. Dengan mengatur parameter seperti metode cross-validation, penggunaan blending, serta paralelisasi proses, class ini memberikan kontrol penuh kepada pengguna dalam menerapkan teknik stacking sesuai kebutuhan. Kegunaan utamanya adalah sebagai alat bantu untuk mengimplementasikan ensemble model dari nol (from scratch) tanpa tergantung pada fungsi otomatis dari pustaka seperti scikit-learn, sehingga cocok untuk pembelajaran konsep maupun eksperimen lanjutan dalam machine learning.

Buat class bernama Stack yang berisi <br>

Attribute:


Method:


In [2]:
# Definisikan class stack
# Definisikan class StackingClassifier (from scratch)
class StackingClassifier:
    def __init__(self, base_models, meta_model, n_folds=5):
        """
        base_models : list
            Daftar model dasar (misal: [DecisionTreeClassifier(), KNeighborsClassifier()])
        meta_model : object
            Model meta yang akan dilatih pada prediksi dari base models
        n_folds : int
            Jumlah fold untuk cross-validation (default = 5)
        """
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    def fit(self, X, y):
        """Melatih base models dan meta model menggunakan skema out-of-fold."""
        from sklearn.model_selection import KFold
        self.base_models_ = [list() for _ in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)

        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            for train_idx, holdout_idx in kfold.split(X, y):
                instance = clone(model)
                instance.fit(X[train_idx], y[train_idx])
                y_pred = instance.predict(X[holdout_idx])
                out_of_fold_predictions[holdout_idx, i] = y_pred
                self.base_models_[i].append(instance)

        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    def predict(self, X):
        """Melakukan prediksi dengan menggabungkan hasil prediksi base models menggunakan meta model."""
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)


### **Upload datasets**

In [3]:
# Load dataset iris dari sklearn
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
glass_df = iris.frame

# Pisahkan fitur dan target
X1 = glass_df.drop(columns=["target"])
y1 = glass_df["target"]

# Split data menjadi train dan test
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

print("Data train:", X1_train.shape)
print("Data test :", X1_test.shape)

Data train: (120, 4)
Data test : (30, 4)


# **Model training and evaluation of the obtained results**
Both in the case of classification and regression, stacking and blending showed the same and not the best results. As a rule, this situation occurs for two reasons: given that metadata is based on predictions of basic models, the presence of weak basic models can reduce the accuracy of stronger ones which will reduce the final prediction as a whole. Also a small amount of training data often leads to overfitting which in turn reduces the accuracy of predictions.

In this case the problem can be partially solved by setting stack_method='predict_proba' when each basic classifier outputs class membership probabilities instead of the classes themselves which can help increase accuracy in the case of non-mutually exclusive classes. Also this method works better with noise in the data. As you can see this method has significantly increased the accuracy of the model. With the right selection of models and hyperparameters the accuracy will be even higher.

Most often stacking shows slightly better results than blending due to the use of k-fold cross-validation but usually the difference is noticeable only on a large amount of data.

In [4]:
# StackingClassifier dan Blending Classifier

# STACKING CLASSIFIER (from scratch)
from sklearn.base import clone

class StackingClassifier:
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    def fit(self, X, y):
        from sklearn.model_selection import KFold
        self.base_models_ = [list() for _ in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)

        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            for train_idx, holdout_idx in kfold.split(X, y):
                instance = clone(model)
                instance.fit(X.iloc[train_idx], y.iloc[train_idx])
                y_pred = instance.predict(X.iloc[holdout_idx])
                out_of_fold_predictions[holdout_idx, i] = y_pred
                self.base_models_[i].append(instance)

        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)

# BLENDING CLASSIFIER (versi sederhana)
class BlendingClassifier:
    def __init__(self, base_models, meta_model, holdout_size=0.2):
        self.base_models = base_models
        self.meta_model = meta_model
        self.holdout_size = holdout_size

    def fit(self, X, y):
        X_train, X_holdout, y_train, y_holdout = train_test_split(
            X, y, test_size=self.holdout_size, random_state=42
        )

        self.base_models_ = [clone(model).fit(X_train, y_train) for model in self.base_models]

        holdout_predictions = np.column_stack([
            model.predict(X_holdout) for model in self.base_models_
        ])

        self.meta_model_ = clone(self.meta_model).fit(holdout_predictions, y_holdout)
        return self

    def predict(self, X):
        meta_features = np.column_stack([
            model.predict(X) for model in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)


**StackingClassifier (scikit-learn)**


* Merupakan implementasi resmi stacking untuk klasifikasi dalam scikit-learn.

* Mempermudah proses ensemble dengan base learners dan final estimator (meta learner) dalam satu objek.

* Sudah menangani cross-validation secara internal sehingga mengurangi risiko data leakage.

* Mendukung parameter passthrough=True jika ingin menggabungkan fitur asli dengan meta-features.

* Cocok untuk digunakan dalam pipeline dan produksi karena stabil dan teruji.

In [5]:
# StackingClassifier (scikit-learn)

# STACKING CLASSIFIER (Menggunakan Scikit-Learn)
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

sk_estimators = [
    ('dt', DecisionTreeClassifier(random_state=0)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

meta_model = LogisticRegression()

stack_methods = ['predict', 'predict_proba']

for i, method in enumerate(stack_methods):
    print(f"\n=== STACKING METHOD {i+1}: '{method}' ===")

    sk_stack = StackingClassifier(
        estimators=sk_estimators,
        final_estimator=meta_model,
        stack_method=method,
        passthrough=False,
        cv=5,
        n_jobs=-1
    )

    sk_stack.fit(X1_train, y1_train)

    y_pred = sk_stack.predict(X1_test)
    acc = accuracy_score(y1_test, y_pred)
    print("Akurasi :", round(acc, 4))



=== STACKING METHOD 1: 'predict' ===
Akurasi : 1.0

=== STACKING METHOD 2: 'predict_proba' ===
Akurasi : 1.0
